In [ ]:
from jpype import * 
import glob

In [ ]:
filenames = glob.glob('/home/miotakei/Applications/pgx-19.1.0/lib/*')

In [ ]:
pgx_jar_classpath = ':'.join(filenames)

In [ ]:
# start JVM (any other param can be added like TrustStore, KeyStore etc.
startJVM(getDefaultJVMPath(), "-ea", "-Djava.class.path=" + pgx_jar_classpath )

pgxClass = JClass('oracle.pgx.api.Pgx')

In [ ]:
# create a session on a PGX server
session = pgxClass.createSession('http://localhost:7007', 'session1')

# load the graph from disk with a JSON file
# important: Json file is accessible by python, graph data file must be accessible by the PGX server


In [ ]:
graph = session.readGraphWithProperties("<path of json file>")

In [ ]:
analyst = session.createAnalyst()

In [ ]:
pgxResultSetNode = graph.queryPgql(" \
  SELECT id(n), n.name, n.prob, e.times \
  MATCH (n) \
              , (x)-[e]->(y)  \
  WHERE e.times >= (SELECT e.times MATCH (n) WHERE max) \
  AND ((n) = (x) OR (n) = (y)) \
  ORDER BY e.times DESC \
")

In [ ]:
pgxResultSetEdge = graph.queryPgql(" \
  SELECT id(x), id(y), e.times \
  MATCH (x)-[e]->(y) \
  WHERE e.times >= 234 \
")

In [ ]:
it_node = pgxResultSetNode.getResults().iterator()
dict_node = [{"id":i.get(0),"label":i.get(1), "value":i.get(2)} for i in it_node]
# 重複削除
dict_node = [dict(t) for t in set([tuple(d.items()) for d in dict_node])]

In [ ]:
it_edge = pgxResultSetEdge.getResults().iterator()
dict_edge = [{"from":i.get(0),"to":i.get(1)} for i in it_edge]

In [ ]:
from IPython.display import Javascript
import json

# Transform the graph into a JSON graph
data = {"nodes":dict_node, "edges":dict_edge}
jsonGraph = json.dumps(data, indent=4)

# Send to Javascript
Javascript("""window.jsonGraph={};""".format(jsonGraph))

In [ ]:
%%html
<div id="mynetwork"></div>

In [ ]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis){
    
    // create a network
    var container = document.getElementById('mynetwork');
    var options = {
        width: '800px',
        height: '800px'
    };
    
    // We load the JSON graph we generated from iPython input
    var graph = window.jsonGraph;
    
    // Display Graph
    var network = new vis.Network(container, graph, options);
});

In [ ]:
#shutdownJVM()
shutdownJVM()